In [1]:
# import sys
# print(sys.version)
# %pip install numpy
# %pip install matplotlib
# %pip install pandas
# %pip install scikit-spatial
# %pip install PyQt5

In [2]:
# Librerias
import numpy as np
from matplotlib import use
import matplotlib.pyplot as plt
import pandas as pd
from skspatial.objects import Plane, Point, Vector, Line
from skspatial.plotting import plot_3d


use('Qt5Agg')

In [3]:
inter = None
color_t = None
caras = None
vertex = None
normal = None
M = None
A = None
mp = None
eigenvalues = None
eigenvectors = None
traces = None
tri = None
color_m = None
size = None
max_value = 0
x_axis = None
y_axis = None
squares = None
M_size = None

# FRAC0006_nrIter27.txt
# FRAC0003_nrIter4.txt
# FRAC0019_nrIter27.txt

filename = 'FRAC0003_nrIter4.txt'


#### 1. Cunstruir caja para un conjunto de puntos en 3D

In [4]:
def new_plot():
    fig = plt.figure()
    return fig.add_subplot(111, projection='3d')

In [5]:
def build_box(axis):
        lines = [
            [0, 1], [0, 2], [0, 4],
            [1, 3], [1, 5], [2, 3],
            [2, 6], [3, 7], [4, 5],
            [4, 6], [5, 7], [6, 7]
        ]
        for line in lines:
            axis.plot(
                [vertex[line[0], 0], vertex[line[1], 0]],
                [vertex[line[0], 1], vertex[line[1], 1]],
                [vertex[line[0], 2], vertex[line[1], 2]],
                color='black'
            )

In [6]:
def config_plot(axis, title):
    axis.set_xlabel('X')
    axis.set_ylabel('Y')
    axis.set_zlabel('Z')
    plt.title(title)
    plt.xlim(x_axis - max_value, x_axis + max_value)
    plt.ylim(y_axis - max_value, y_axis + max_value)
    #plt.show()

In [7]:
# Carga los datos de la fractura desde un archivo de texto.
dataset = []
with open(filename, 'r') as file:
    for line in file:
        values = [float(value) for value in line.split()]
        dataset.append(values)
M = np.array(dataset)
M_size = M.shape

In [8]:
def limits():
    global x_axis, y_axis, max_value
    min_limits = np.min(M, axis=0)
    max_limits = np.max(M, axis=0)

    for i in range(3):
        max_value = max(max_value, abs(max_limits[i] - min_limits[i]))
    max_value/=2
    x_axis = np.min(M[:, 0]) + max_value/2
    y_axis = np.min(M[:, 1]) + max_value/2
    # print(x_axis, y_axis, max_value)
    # print(x_axis - max_value, x_axis + max_value)
    # print(y_axis - max_value, y_axis + max_value)
limits()

In [9]:
"""
    Calcula la matriz A (matriz de covarianza) de una matriz M y
    los autovalores y autovectores
"""
def load_matrix():
    global A, mp, M

    A = np.dot(M.T, M)

    mp = np.mean(M, axis=0)

    eigenvalues, eigenvectors = np.linalg.eig(A)

load_matrix()

In [10]:
"""
    Calcula los vértices de la caja que encierra la fractura (matriz m)
"""
def get_box():

    global vertex, M, mp
    points = M - mp
    eigenvalues, _eigenvectors = np.linalg.eig(np.cov(points, rowvar=False))
    points = np.dot(points, _eigenvectors)

    # Calcula los límites de la caja en el nuevo sistema de coordenadas
    min_limits = np.min(points, axis=0)
    max_limits = np.max(points, axis=0)

    # Crear puntos de los límites de la caja en el sistema transformado
    vertex = np.array(np.meshgrid([min_limits[0], max_limits[0]], [min_limits[1], max_limits[1]],
                                        [min_limits[2], max_limits[2]])).T.reshape(-1, 3)

    # Transformar los vértices de la caja de vuelta al sistema original
    vertex = np.dot(vertex, _eigenvectors.T) + mp

get_box()

#### 2. Desplegar los puntos y desplegar los autovectores

In [11]:
def color_points():
    global color_m, size
    color_m = []
    size = int(len(M) / 20)
    color_m = np.random.rand(size, 3)


color_points()

#### 3. Desplegar la caja contenedora

In [12]:
"""
    Representación de los datos en 3D utilizando Matplotlib
"""
def print_points():
    axis = new_plot()
    build_box(axis)
    
    for i in range(size):
        m = M[i * 20:i * 20 + 20]
        axis.scatter(m[:, 0], m[:, 1], m[:, 2], color=color_m[i], s=2)

    config_plot(axis, "Points")

#### 4. Calculo de los triangulos y las normales

In [13]:
"""
    Realizar la triangulación de los puntos, teniento en cuenta que cada 20 puntos se apueden realizar
    triangulaiones con los siguientes 20 puntos.

    la variable tri alamacena la dirección de los puntos que forman un triangulo
"""
def triangularization():

    global color_t, tri
    color_t = []
    tri = []
    for i in range(0, size - 1):
        for j in range(20):
            if j == 19:
                tri.append([j + (20 * i), j + (20 * (i + 1)), 0 + (20 * (i + 1))])
                tri.append([j + (20 * i), 0 + (20 * (i + 1)), 0 + (20 * i)])
            else:
                tri.append([j + (20 * i), j + (20 * (i + 1)), (j + 1) + (20 * (i + 1))])
                tri.append([j + (20 * i), (j + 1) + (20 * (i + 1)), (j + 1) + (20 * i)])

            color = (color_m[i] + 2 * color_m[i + 1]) / 3
            color_t.append(color)
            color = (2 * color_m[i] + color_m[i + 1]) / 3
            color_t.append(color)
    color_t = np.array(color_t)
    tri = np.array(tri)

triangularization()

In [14]:
"""
    graficar triangularizacion
"""
def print_triangles():
    axis = new_plot()
    build_box(axis)
    
    for s in range(len(tri)):
        triangle = np.array([M[tri[s][0]], M[tri[s][1]], M[tri[s][2]], M[tri[s][0]]])
        axis.plot(triangle[:, 0], triangle[:, 1], triangle[:, 2], color=color_t[s])

    config_plot(axis, "Triangles")

In [15]:
"""
sacar la norma de cada triangulo
"""
isdegenerate = []
def norm():
    global normal
    normal = np.zeros(((size - 1)*40, 3))
    aux = 1
    i = 0
    for triangle in tri:
        p1, p2, p3 = M[triangle]
        v1 = p2 - p1
        v2 = p3 - p1
        n = np.cross(v1, v2)
        normal[i] = n / np.linalg.norm(n)

        if np.isnan(normal[i]).any():
            isdegenerate.append(i)
            
            random_vector = np.random.rand(3)

            random_vector /= np.linalg.norm(random_vector)

            normal[i] = aux * normal[i - 1] / np.abs(np.dot(random_vector, normal[i - 1]))

            normal[i] /= np.linalg.norm(normal[i])

        if i > 0:
            aux = np.dot(normal[i], normal[i - 1])
        i += 1
    normal = np.array(normal)

#  editar en fracture
norm()

In [16]:
"""
graficar la norma de los triangulos
"""
def print_norm():
    axis = new_plot()
    build_box(axis)
    
    for i in range(len(normal)):
        mp = np.mean(M[tri[i]], axis=0)
        v = 0.3 * normal[i]
        axis.quiver(mp[0], mp[1], mp[2], v[0], v[1], v[2], color='b')

    config_plot(axis, "Normal of the triangles")

In [17]:
print_points()
print_triangles()
print_norm()
plt.show()

In [18]:
normal_vertex = None

def get_normal_vertex():
    global normal_vertex
    normal_vertex = [[] for _ in range(M.shape[0])]
    for i, n in enumerate(normal):
        for j in tri[i]:
            normal_vertex[j].append(n)

    for i in range(len(normal_vertex)):
        normal_vertex[i] = np.mean(normal_vertex[i], axis=0)

    normal_vertex = np.array(normal_vertex)

get_normal_vertex()



#### 5. Interseccion normales del triangulo con las caras de la caja

In [19]:
caras = []
points = np.array(
    [[0, 1, 2, 3],
    [0, 1, 4, 5],
    [0, 4, 2, 6],
    [7, 5, 3, 1],
    [7, 6, 3, 2],
    [7, 6, 5, 4]]
    )

def plane_from_points():
    """
        Calcula los planos a partir de puntos dados y los guarda en la lista.
    """
    for p in points:
        p1, p2, p3 = vertex[p[0]], vertex[p[1]], vertex[p[2]]
        caras.append(Plane.from_points(p1,p2,p3))

plane_from_points()

In [20]:
"""
    Crea una línea a partir de un punto y un vector.

    Args:
    - point: Punto inicial de la línea.
    - vector: Vector director de la línea.

    Returns:
    - Líne
"""
def make_line(point, vector):
    return Line(point=point, direction=vector)

In [21]:
"""
Verifica si un punto final está en la dirección de un vector dado desde un punto inicial.
"""
def is_direction(pi, pf, vd):
    vr = pf - pi
    return np.dot(vr, vd) > 0


In [22]:
"""
Encuentra la intersección más cercana a un punto inicial desde una lista de intersecciones.
"""
def close_intersection(pi, intersections):
    intersect = None
    min_dist = float('inf')
    point = Point(pi)
    for intersection in intersections:
        dist = point.distance_point(intersection)
        if dist < min_dist:
            min_dist = dist
            intersect = intersection
            
    return intersect

In [23]:
"""
Encuentra la intersección más cercana de una línea con varias caras (planos).
"""
def intersection_plane_line(line, pi, vd):
    intersections = []
    for i, plane in enumerate(caras):
        point = plane.intersect_line(line)
        if point is not None and is_direction(pi, point, vd):
            intersections.append(point)

    return close_intersection(pi, intersections)


In [24]:
"""
Calcula las intersecciones más cercanas de líneas definidas por puntos medios de triángulos y sus normales con las caras de la caja.

Utiliza las normales de los triángulos y los puntos medios de los triángulos para definir las líneas.
Luego, encuentra la intersección más cercana de cada línea con las caras de la caja.
Las intersecciones se almacenan en una variable global `inter`.
"""
def intersection():
    global inter  
    inter = [] 
    
    for i in range(normal_vertex.shape[0]):
        mp = M[i] # np.mean(M[tri[i]], axis=0)
        v = normal_vertex[i]
        if np.any(np.isnan(normal_vertex[i])):
            inter.append(normal_vertex[i])
            continue
        line = make_line(mp, v)
        r = intersection_plane_line(line, mp, v)
        inter.append(r)
    inter = np.array(inter)


intersection()

In [25]:
# def refactor_inter():
#     global inter
#     inter_tmp = []
#     for i in range(int(len(inter)/40)):
#         tmp = inter[i*40:i*40+40]
        
#         # Separar los elementos en índices impares y pares
#         odd_indices = [tmp[j] for j in range(len(tmp)) if j % 2 != 0]
#         even_indices = [tmp[j] for j in range(len(tmp)) if j % 2 == 0]
        
#         # Agregar primero los impares y luego los pares a inter_tmp
#         inter_tmp.extend(odd_indices)
#         inter_tmp.extend(even_indices)

#     inter = np.array(inter_tmp)

# refactor_inter()

In [26]:
"""
    Añade los puntos de intersección al gráfico.
"""
def print_intersection():
    global color_m
    axis = new_plot()
    build_box(axis)

    for i in range(size):
        m = inter[i * 20:i * 20 + 20]
        axis.scatter(m[:, 0], m[:, 1], m[:, 2], color=color_m[i], s=2)
    #axis.scatter(inter[:, 0], inter[:, 1], inter[:, 2], color=color_m, s=2)
    
    config_plot(axis, "Intersection")

#### 6. triangulos en la superficie de la cara de la caja

In [27]:
def plot_surface_triangles():
    axis = new_plot()
    build_box(axis)
    
    for s in range(len(tri)):
        triangle = np.array([inter[tri[s][0]], inter[tri[s][1]], inter[tri[s][2]], inter[tri[s][0]]])
        axis.plot(triangle[:, 0], triangle[:, 1], triangle[:, 2], color=color_t[s])

    config_plot(axis, "Triangles")

In [38]:
print_intersection()
plot_surface_triangles()
plt.show()

#### No necesario por el momento

In [29]:
# def print_intersection_tmp():
#     global color_t, inter
#     inter_ = inter[:20]
#     axis = new_plot()
#     build_box(axis)
#     for i in range(len(inter_)):
#         if i%2 == 0:
#             axis.scatter(inter_[i][0], inter_[i][1], inter_[i][2], color='r', s=2)
#         else:
#             axis.scatter(inter_[i][0], inter_[i][1], inter_[i][2], color='b', s=2)
        
#     #axis.scatter(inter[:, 0], inter[:, 1], inter[:, 2], color=color_t, s=2)
    
#     config_plot(axis, "Intersection")
# print_intersection_tmp()
# plt.show()
    

In [30]:
# """
#     Calcula los cuadrados a partir de los puntos de intersección y los guarda en la lista squares.
# """
# def get_square():
#     global squares
#     squares = []
#     for i in range(len(inter) - 20):
#         if (i+1) % 20 == 0:
#             squares.append(np.array([i, i-19, i+20, i+1]))
#         else:
#             squares.append(np.array([i, i+1, i+20, i+21]))

#     squares = np.array(squares)

# get_square()

In [31]:
# def get_square():
#     global squares
#     sq = []
#     for i in range(0, len(inter), 2):
#         if (i + 2) % 40 == 0:
#             square = [i, i+1, i+2-40, i+3-40]
#         else:
#             square = [i, i+1, i+2, i+3]
#         sq.append(square)
#     sq = np.array(sq)
    
#     sq2 = []
#     for i in range(41, len(inter), 2): 
#         if (i+1) % 40 == 0:
#             sq2.append([i, i-41, i-38, i-79])
#         else:
#             sq2.append([i, i-41, i+2, i-39])
#     sq2= np.array(sq2)
#     squares = sq[:20]
#     for i in range(1, int(len(sq)/20)):
#         squares = np.concatenate((squares, sq[i*20:i*20+20], sq2[:i*20]))

In [32]:
# """
#     Calcula el color de los cuadrados y los guarda en la lista color_s.
# """
# color_s = []
# def color_square():
#     global color_s
#     for i in squares:
#         color = (color_t[i[0]] + color_t[i[1]] + color_t[i[2]] + color_t[i[3]]) / 4
#         color_s.append(color)
#     color_s = np.array(color_s)
# color_square()

In [33]:
# def print_squares(axis):
#     """
#         Grafica los cuadrados.
#     """
#     for square in squares:
#         s = [inter[square[0]], inter[square[1]], inter[square[3]], inter[square[2]], inter[square[0]]]
#         axis.plot([v[0] for v in s], [v[1] for v in s], [v[2] for v in s], color="b")

In [34]:
# mp_s = []
# def square_prom():
#     """
#         Calcula el punto promedio de cada cuadrado y los guarda en la lista mp_s.
#     """
#     global mp_s
#     for i in squares:
#         mp = np.mean(inter[i], axis=0)
#         mp_s.append(mp)
#     mp_s = np.array(mp_s)

# square_prom()

In [35]:
# def print_p_square():
#     """
#         Grafica los puntos promedio de los cuadrados.
#     """

#     axis = new_plot()
#     build_box(axis)
#     print_squares(axis)
#     axis.scatter(mp_s[:, 0], mp_s[:, 1], mp_s[:, 2],s=1, color='r')
#     config_plot(axis, "Squares")

In [36]:
# print_intersection()
# # print_p_square()
# plt.show()